In [39]:
import tensorflow as tf
import pandas as pd

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [40]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']

def load_data(label_name='Species'):
    """Parses the csv file in TRAIN_URL and TEST_URL."""
    train_path = tf.keras.utils.get_file(fname=TRAIN_URL.split('/')[-1],
                                         origin=TRAIN_URL)
    train = pd.read_csv(filepath_or_buffer=train_path,
                        names=CSV_COLUMN_NAMES,  # list of column names
                        header=0  # ignore the first row of the CSV file.
                       )
    train_label = train.pop(label_name)
    train_features = train
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)
    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    
    test_label = test.pop(label_name)
    test_features = test

    return (train_features, train_label), (test_features, test_label)

In [41]:
(train_feature, train_label), (test_feature, test_label) = load_data()

In [5]:
my_feature_columns = [
    tf.feature_column.numeric_column(key='SepalLength'),
    tf.feature_column.numeric_column(key='SepalWidth'),
    tf.feature_column.numeric_column(key='PetalLength'),
    tf.feature_column.numeric_column(key='PetalWidth')
]

In [46]:

dataset = tf.data.Dataset.from_tensor_slices((dict(train_feature),train_label))
print(dataset)

<TensorSliceDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}, tf.int64)>


In [6]:
def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(buffer_size=1000).repeat(count=None).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [11]:
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        hidden_units=[10, 10],
        n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ll/x0sd639565s_1cgb0v__hvmr0000gn/T/tmpx022t6sc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11aecb898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [38]:
classifier.train(
    input_fn = lambda:train_input_fn(train_feature, train_label, 50),
    steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/ll/x0sd639565s_1cgb0v__hvmr0000gn/T/tmpx022t6sc/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10001 into /var/folders/ll/x0sd639565s_1cgb0v__hvmr0000gn/T/tmpx022t6sc/model.ckpt.
INFO:tensorflow:loss = 0.35742885, step = 10001
INFO:tensorflow:global_step/sec: 776.965
INFO:tensorflow:loss = 0.8062043, step = 10101 (0.130 sec)
INFO:tensorflow:global_step/sec: 1354.98
INFO:tensorflow:loss = 4.071093, step = 10201 (0.074 sec)
INFO:tensorflow:global_step/sec: 1371.59
INFO:tensorflow:loss = 0.9351113, step = 10301 (0.073 sec)
INFO:tensorflow:global_step/sec: 1287.25
INFO:tensorflow:loss = 1.0137123, step = 10401 (0.078 sec)
INFO:tensorflow:global_step/sec: 1336.18
INFO:tensorflow:loss = 0.5702748

INFO:tensorflow:global_step/sec: 1418.1
INFO:tensorflow:loss = 0.67647636, step = 17801 (0.071 sec)
INFO:tensorflow:global_step/sec: 1407.24
INFO:tensorflow:loss = 0.66325164, step = 17901 (0.071 sec)
INFO:tensorflow:global_step/sec: 1379.88
INFO:tensorflow:loss = 2.899674, step = 18001 (0.072 sec)
INFO:tensorflow:global_step/sec: 1363.08
INFO:tensorflow:loss = 0.8986403, step = 18101 (0.073 sec)
INFO:tensorflow:global_step/sec: 1344.32
INFO:tensorflow:loss = 0.9294461, step = 18201 (0.074 sec)
INFO:tensorflow:global_step/sec: 1365.15
INFO:tensorflow:loss = 0.6085691, step = 18301 (0.073 sec)
INFO:tensorflow:global_step/sec: 1355.38
INFO:tensorflow:loss = 2.219574, step = 18401 (0.074 sec)
INFO:tensorflow:global_step/sec: 1348.87
INFO:tensorflow:loss = 0.56057435, step = 18501 (0.074 sec)
INFO:tensorflow:global_step/sec: 1341.54
INFO:tensorflow:loss = 2.789414, step = 18601 (0.075 sec)
INFO:tensorflow:global_step/sec: 1346.7
INFO:tensorflow:loss = 0.9683424, step = 18701 (0.074 sec)
IN

In [32]:
def test_input_fn(features, labels, batch_size):
    features = dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)
        
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    
    return dataset.make_one_shot_iterator().get_next()
    

In [36]:
print(dict(train_feature))

{'SepalLength': 0      6.4
1      5.0
2      4.9
3      4.9
4      5.7
5      4.4
6      5.4
7      6.9
8      6.7
9      5.1
10     5.2
11     6.9
12     5.8
13     5.4
14     7.7
15     6.3
16     6.8
17     7.6
18     6.4
19     5.7
20     6.7
21     6.4
22     5.4
23     6.1
24     7.2
25     5.2
26     5.8
27     5.9
28     5.4
29     6.7
      ... 
90     6.5
91     6.1
92     5.1
93     4.6
94     6.5
95     4.6
96     4.6
97     7.7
98     5.9
99     5.1
100    4.9
101    4.9
102    4.5
103    5.8
104    5.0
105    5.2
106    5.3
107    5.0
108    5.6
109    4.8
110    6.3
111    5.7
112    5.0
113    6.3
114    5.0
115    5.5
116    5.7
117    4.4
118    4.8
119    5.5
Name: SepalLength, Length: 120, dtype: float64, 'SepalWidth': 0      2.8
1      2.3
2      2.5
3      3.1
4      3.8
5      3.2
6      3.4
7      3.1
8      3.1
9      3.7
10     2.7
11     3.1
12     4.0
13     3.9
14     3.8
15     3.3
16     3.2
17     3.0
18     3.2
19     4.4
20     3.3
21     2.8
22     3.

In [33]:
eval_result = classifier.evaluate(input_fn=lambda:eval_input_fn(test_feature, test_label, 50))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-23-11:22:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/ll/x0sd639565s_1cgb0v__hvmr0000gn/T/tmpx022t6sc/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-23-11:22:56
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.96666664, average_loss = 0.058339145, global_step = 10000, loss = 1.7501744
